<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导入依赖" data-toc-modified-id="导入依赖-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入依赖</a></span></li><li><span><a href="#数据加工" data-toc-modified-id="数据加工-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>数据加工</a></span><ul class="toc-item"><li><span><a href="#加载" data-toc-modified-id="加载-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>加载</a></span></li><li><span><a href="#缺失值处理" data-toc-modified-id="缺失值处理-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>缺失值处理</a></span></li><li><span><a href="#类别特征" data-toc-modified-id="类别特征-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>类别特征</a></span></li><li><span><a href="#过采样" data-toc-modified-id="过采样-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>过采样</a></span></li><li><span><a href="#采样后样本" data-toc-modified-id="采样后样本-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>采样后样本</a></span></li><li><span><a href="#需分段连续/数值特征处理" data-toc-modified-id="需分段连续/数值特征处理-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>需分段连续/数值特征处理</a></span></li><li><span><a href="#用户和商品（稀疏）类别特征编码" data-toc-modified-id="用户和商品（稀疏）类别特征编码-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>用户和商品（稀疏）类别特征编码</a></span></li><li><span><a href="#类别特征embedding准备" data-toc-modified-id="类别特征embedding准备-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>类别特征embedding准备</a></span></li></ul></li><li><span><a href="#训练和评估样本" data-toc-modified-id="训练和评估样本-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>训练和评估样本</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#模型加载" data-toc-modified-id="模型加载-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>模型加载</a></span></li><li><span><a href="#模型预测" data-toc-modified-id="模型预测-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>模型预测</a></span><ul class="toc-item"><li><span><a href="#商品库（去重）" data-toc-modified-id="商品库（去重）-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>商品库（去重）</a></span></li><li><span><a href="#商品库样本" data-toc-modified-id="商品库样本-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>商品库样本</a></span></li><li><span><a href="#用户评估样本和商品库样本-模型结果预测" data-toc-modified-id="用户评估样本和商品库样本-模型结果预测-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>用户评估样本和商品库样本 模型结果预测</a></span></li><li><span><a href="#相似向量检索" data-toc-modified-id="相似向量检索-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>相似向量检索</a></span><ul class="toc-item"><li><span><a href="#L2距离（欧式）" data-toc-modified-id="L2距离（欧式）-6.4.1"><span class="toc-item-num">6.4.1&nbsp;&nbsp;</span>L2距离（欧式）</a></span></li><li><span><a href="#cos距离" data-toc-modified-id="cos距离-6.4.2"><span class="toc-item-num">6.4.2&nbsp;&nbsp;</span>cos距离</a></span></li></ul></li></ul></li><li><span><a href="#评估" data-toc-modified-id="评估-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>评估</a></span><ul class="toc-item"><li><span><a href="#评估实际购买过商品用户" data-toc-modified-id="评估实际购买过商品用户-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>评估实际购买过商品用户</a></span></li><li><span><a href="#用户实际购买商品" data-toc-modified-id="用户实际购买商品-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>用户实际购买商品</a></span></li></ul></li><li><span><a href="#存在的问题/可改进的地方" data-toc-modified-id="存在的问题/可改进的地方-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>存在的问题/可改进的地方</a></span></li><li><span><a href="#参考" data-toc-modified-id="参考-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>参考</a></span></li></ul></div>

### 导入依赖

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from collections import defaultdict
from tensorflow.keras.preprocessing.sequence import pad_sequences

import os
import sys
# sys.path.append('/Users/lxh/Desktop/recommend-tf2.0/src')
import numpy as np
import faiss
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from model import Dssm, DNN

### 数据加工

#### 加载 

In [2]:
df = pd.read_csv('recall_sample2.csv')
df.shape

(2387524, 64)

#### 缺失值处理

In [3]:
null_feture_col = {'last_total_ride_card_times':0,'last_ride_card_vaild_days':0,'last_original_price':0,'last_pay_amt':0,
    'last_coupon_product_id':0,'last_coupon_product_name':'0','card_7d':0,'card_30d':0,'card_60d':0,
    'mean_total_ride_card_times_30d':0.,'mean_ride_card_vaild_days_30d':0.,'mean_original_price_60d':0.,
    'mean_pay_amt_60d':0.,'max_original_price_60d':0,'max_pay_amt_60d':0,'product_id':0,'product_name':'0',
    'original_price':0,'during_days':0,'discount_times_day':0,'discount_all':0,'discount_amount':0,
    'discount_rule':0}
df = df.fillna(null_feture_col)

#### 类别特征

In [4]:
# 无需处理的用户类别特征
user_noneed_X_cols = ['gender','city_id','register_y','register_m','register_d','register_dw','is_idcard',
                'client_type','sys_ver','is_open_wepay_score','last_total_ride_card_times','last_ride_card_vaild_days',
               'card_7d','card_30d','card_60d','order_days_7d','order_days_30d']
# 无需处理的商品类别特征
item_noneed_X_cols = ['during_days','discount_times_day','discount_all','discount_amount','discount_rule']

# 需分段的数值特征
user_need_X_cols = ['age','register_days','active_count_7d','active_count_30d','active_count_180d',
               'order_count_7d','order_count_30d','order_count_t']

#### 过采样

In [6]:
#### 商品总抵扣次数正负样本量对比

df.groupby(['td_is_buy_ride_card','discount_all']).user_id.count()

td_is_buy_ride_card  discount_all
0                    2               70779
                     3               72248
                     4               70504
                     5               71813
                     6               76017
                     7               72007
                     8               73065
                     9               72340
                     10              71850
                     12              78078
                     14              74776
                     15              70612
                     16              78753
                     18              58255
                     20              71929
                     24              82773
                     25              39887
                     28              67164
                     30              71974
                     35              48247
                     40              86667
                     45              71704
                    

In [7]:
# 样本过采样，总次数1～100复制100次，101～1000复制10次，其余不变
df['group_total_cnt'] = df.groupby(['td_is_buy_ride_card','discount_all'])['user_id'].transform('count')
sample_low = df[df.group_total_cnt.between(1,100)]
sample_med = df[df.group_total_cnt.between(101,1000)]
sample_high = df[df.group_total_cnt.between(1001,np.inf)]
print(sample_low.shape,sample_med.shape,sample_high.shape)

print('before',sample_low.shape)
df_low = pd.DataFrame()
for _ in range(100):
    df_low = pd.concat([df_low, sample_low])

df_low = df_low.reset_index(drop=True)
print('after:',df_low.shape)

print('before',sample_med.shape)
df_med = pd.DataFrame()
for _ in range(10):
    df_med = pd.concat([df_med, sample_med])

df_med = df_med.reset_index(drop=True)
print('after:',df_med.shape)

(412, 65) (1350, 65) (2385762, 65)
before (412, 65)
after: (41200, 65)
before (1350, 65)
after: (13500, 65)


#### 采样后样本


In [8]:
df_sampled = pd.concat([df_low, df_med,sample_high]).reset_index(drop=True)
df_sampled.shape

(2440462, 65)

#### 需分段连续/数值特征处理

In [9]:
# 数值区间探索EDA
for col in user_need_X_cols:
#     print(col,df_sampled[col].value_counts())
    print(col,df_sampled[col].median(),df_sampled[col].quantile(0.5),df_sampled[col].quantile(0.75),df_sampled[col].quantile(0.25))
    print('=================')

age 33.0 33.0 41.0 24.0
register_days 835.0 835.0 1179.0 405.0
active_count_7d 106.0 106.0 351.0 0.0
active_count_30d 388.0 388.0 1138.0 91.0
active_count_180d 1261.0 1261.0 3634.0 368.0
order_count_7d 1.0 1.0 4.0 0.0
order_count_30d 5.0 5.0 14.0 1.0
order_count_t 57.0 57.0 176.0 15.0


In [10]:
# 粗分10段
for col in user_need_X_cols[1:]:
    df_sampled['re_'+col]=pd.qcut(df_sampled[col],10,duplicates='drop')
# 年龄较细致分7段
df_sampled['re_age'] = pd.cut(df_sampled['age'], bins=[-1, 18, 25, 35, 45, 60, 100,9999 ], labels=['0-15', '15-25', '25-35','35-45', '45-60', '60-100', '100-'])



#### 用户和商品（稀疏）类别特征编码

In [11]:
user_sparse_features = ['re_'+col for col in user_need_X_cols] + user_noneed_X_cols

item_features = item_noneed_X_cols
    

In [12]:
feature_max_idx = {}
lbe_list = []
# 类别特征编码
for feature in user_sparse_features + item_features:
    print(feature)
    lbe = LabelEncoder()
    df_sampled[feature] = lbe.fit_transform(df_sampled[feature])
    lbe_list.append(lbe)
    # 记录每个类别特征最大取值数量
    feature_max_idx[feature] = df_sampled[feature].max() + 1
feature_max_idx

re_age
re_register_days
re_active_count_7d
re_active_count_30d
re_active_count_180d
re_order_count_7d
re_order_count_30d
re_order_count_t
gender
city_id
register_y
register_m
register_d
register_dw
is_idcard
client_type
sys_ver
is_open_wepay_score
last_total_ride_card_times
last_ride_card_vaild_days
card_7d
card_30d
card_60d
order_days_7d
order_days_30d
during_days
discount_times_day
discount_all
discount_amount
discount_rule


{'re_age': 7,
 're_register_days': 10,
 're_active_count_7d': 7,
 're_active_count_30d': 9,
 're_active_count_180d': 10,
 're_order_count_7d': 6,
 're_order_count_30d': 8,
 're_order_count_t': 10,
 'gender': 3,
 'city_id': 825,
 'register_y': 7,
 'register_m': 12,
 'register_d': 31,
 'register_dw': 7,
 'is_idcard': 2,
 'client_type': 5,
 'sys_ver': 2,
 'is_open_wepay_score': 2,
 'last_total_ride_card_times': 30,
 'last_ride_card_vaild_days': 18,
 'card_7d': 10,
 'card_30d': 22,
 'card_60d': 40,
 'order_days_7d': 8,
 'order_days_30d': 31,
 'during_days': 9,
 'discount_times_day': 29,
 'discount_all': 29,
 'discount_amount': 2,
 'discount_rule': 2}

#### 类别特征embedding准备

In [13]:
def sparseFeature(feat, feat_num, feat_len=1, embed_dim=4):
    """
    create dictionary for varlen sparse feature
    :param feat: feature name
    :param feat_num: the total number of sparse features that do not repeat
    :param feat_len: while feature is array: feat_len > 1
    :param embed_dim: embedding dimension
    :return:
    """
    return {'feat': feat, 'feat_num': feat_num, 'feat_len': feat_len, 'embed_dim': embed_dim}

# 每个特征组装成字典格式
# embed_dim=16
user_sparse_feat_cols = []
# user_sparse_feat_cols.append(sparseFeature(feat='user_id', feat_num=feature_max_idx['user_id'], embed_dim=embed_dim))
user_sparse_feat_cols = user_sparse_feat_cols + [sparseFeature(feat=uf, feat_num=feature_max_idx[uf]) for uf in user_sparse_features]


item_feat_cols = []
# item_feat_cols.append(sparseFeature(feat='coupon_id', feat_num=feature_max_idx['coupon_id'], embed_dim=embed_dim))
# item_type = ['during_days', 'discount_total', 'original_price']
item_feat_cols = item_feat_cols + [sparseFeature(feat=mt, feat_num=feature_max_idx[mt]) for mt in item_features]



### 训练和评估样本

In [14]:
train, test = train_test_split(df_sampled, test_size=0.1)

train_X = [{feat: train[feat].values for feat in user_sparse_features },
           {feat: train[feat].values for feat in item_features}]

train_y = train['td_is_buy_ride_card'].values.astype('int32')

print(train_X,train_y)

test_X = [{feat: test[feat].values for feat in user_sparse_features  },
          {feat: test[feat].values for feat in item_features}]

test_y = test['td_is_buy_ride_card'].values.astype('int32')


[{'re_age': array([4, 5, 5, ..., 4, 3, 3]), 're_register_days': array([4, 8, 9, ..., 1, 8, 1]), 're_active_count_7d': array([0, 0, 5, ..., 0, 2, 6]), 're_active_count_30d': array([0, 0, 8, ..., 3, 3, 8]), 're_active_count_180d': array([0, 0, 9, ..., 1, 5, 9]), 're_order_count_7d': array([0, 0, 5, ..., 0, 1, 5]), 're_order_count_30d': array([0, 0, 7, ..., 2, 2, 7]), 're_order_count_t': array([1, 0, 8, ..., 0, 7, 7]), 'gender': array([1, 2, 1, ..., 2, 2, 2]), 'city_id': array([640, 130,  57, ...,  11, 235, 314]), 'register_y': array([4, 2, 2, ..., 6, 2, 6]), 'register_m': array([3, 5, 1, ..., 0, 8, 1]), 'register_d': array([21, 28,  7, ..., 30,  6, 10]), 'register_dw': array([3, 2, 4, ..., 4, 5, 6]), 'is_idcard': array([1, 1, 1, ..., 1, 1, 1]), 'client_type': array([2, 2, 2, ..., 0, 2, 2]), 'sys_ver': array([0, 0, 0, ..., 1, 1, 0]), 'is_open_wepay_score': array([0, 0, 1, ..., 1, 1, 1]), 'last_total_ride_card_times': array([ 0,  0, 25, ...,  0,  0, 19]), 'last_ride_card_vaild_days': array

### 模型训练

In [15]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2, 3'
# ========================= Hyper Parameters =======================
embed_dim = 16
dnn_dropout = 0.3
hidden_units = [256, 128, 64]

learning_rate = 0.00005
batch_size = 512
epochs = 5

# ========================== Create dataset =======================
# user_feat_cols,user_dense_feat_cols, item_feat_cols, train_X, train_y, test_X, test_y = create_ml_100k_dataset_test(embed_dim=embed_dim)
user_feat_cols, item_feat_cols = user_sparse_feat_cols,item_feat_cols
# ============================Build Model==========================

model = Dssm(user_sparse_feature_columns=user_feat_cols, \
                 item_sparse_feature_columns=item_feat_cols, dnn_dropout=dnn_dropout)
    # model = dssm.summary()
    # model.summary()
    # ============================Compile============================
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=Adam(learning_rate=learning_rate))
#         loss = sampledsoftmaxloss

# ==============================Fit==============================

from tensorflow.keras.callbacks import TensorBoard

log_dir = 'logs'  # 指定日志文件保存的目录
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    train_X,
    train_y,
    epochs=epochs,
    # callbacks=[EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)],  # checkpoint
#         callbacks=[checkpoint],
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[tensorboard_callback]
)

path = 'saved_model_dssm1'
model.save(path, save_format='tf') 


   

Epoch 1/5
3861/3861 [==============================] - 19s 4ms/step - loss: 1.1412e-06 - val_loss: 6.4135e-08
Epoch 2/5
3861/3861 [==============================] - 15s 4ms/step - loss: 3.5392e-08 - val_loss: 3.0172e-08
Epoch 3/5
3861/3861 [==============================] - 15s 4ms/step - loss: 3.0210e-08 - val_loss: 3.0142e-08
Epoch 4/5
3861/3861 [==============================] - 15s 4ms/step - loss: 3.0206e-08 - val_loss: 3.0142e-08
Epoch 5/5
3861/3861 [==============================] - 15s 4ms/step - loss: 3.0206e-08 - val_loss: 3.0142e-08


INFO:tensorflow:Assets written to: saved_model_dssm1/assets


INFO:tensorflow:Assets written to: saved_model_dssm1/assets


### 模型加载

In [16]:
custom_objects = {"Dssm": Dssm}
loaded_model = tf.keras.models.load_model(path, custom_objects=custom_objects)
model = loaded_model.build_graph()

user_embed_model = Model(inputs=model.user_input, outputs=model.user_embed)
item_embed_model = Model(inputs=model.item_input, outputs=model.item_embed)



### 模型预测
#### 商品库（去重）

In [18]:
item_columns = item_noneed_X_cols+['product_name']
print(item_columns)
items = test[item_columns].drop_duplicates(subset=item_columns).reset_index(drop=True)
items

['during_days', 'discount_times_day', 'discount_all', 'discount_amount', 'discount_rule', 'product_name']


,during_days,discount_times_day,discount_all,discount_amount,discount_rule,product_name
0,6,6,6,1,1,20天8次卡
1,7,6,6,1,1,25天8次卡
2,8,18,18,1,1,30天30次卡
3,3,9,9,1,1,7天12次卡
4,3,10,10,1,1,7天14次卡
...,...,...,...,...,...,...
131,8,28,28,1,1,30天120次卡
132,5,23,23,1,1,15天60次卡
133,2,8,8,0,1,5天10次立减卡
134,0,3,3,1,1,3天5次卡


#### 商品库样本

In [19]:
item_X = [{feat: items[feat].values for feat in item_features}]

#### 用户评估样本和商品库样本 模型结果预测

In [20]:
user_embs = user_embed_model.predict(test_X[0])
item_embs = item_embed_model.predict(item_X)
user_embs = tf.squeeze(user_embs).numpy()
item_embs = tf.squeeze(item_embs).numpy()

5/5 [==============================] - 0s 1ms/step


#### 相似向量检索
##### L2距离（欧式）

In [21]:
###  L2距离
# 商品建索引
index_L2 = faiss.IndexFlatL2(item_embs.shape[1])
# faiss.normalize_L2(item_embs)
index_L2.add(np.array(item_embs))
# faiss.normalize_L2(user_embs)
# 用户向量找最近/相似的26个商品向量，对每个用户返回最近26个商品的距离列表和索引列表
D_L2, I_L2 = index_L2.search(np.ascontiguousarray(user_embs), 26)


In [22]:
# user_embs中index=1的用户 L2 召回商品
for i in I_L2[1]: 
    print(items.loc[i].product_name)

30天45次卡
5天2次卡
4天3次卡
3天2次卡
20天45次卡
5天3次卡
3天3次卡
30天40次卡
5天6次卡
25天45次卡
7天3次立减卡
3天6次卡
30天45次立减卡
7天3次卡
30天15次卡
20天30次卡
30天18次卡
15天30次卡
20天15次卡
30天35次卡
30天6次卡
15天40次卡
4天2次卡
4天4次卡
30天立减卡
7天10次卡


##### cos距离
这个是和我们训练一致的距离度量

In [23]:
# cos距离
index = faiss.IndexFlatIP(item_embs.shape[1])
faiss.normalize_L2(item_embs)
index.add(np.array(item_embs))
faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 26)


In [24]:
# user_embs中index=1的用户 cos 召回商品
for i in I[1]: 
    print(items.loc[i].product_name)

30天45次卡
5天2次卡
4天3次卡
3天3次卡
3天2次卡
5天3次卡
25天45次卡
20天45次卡
20天12次卡
30天40次卡
30天18次卡
5天6次卡
30天30次卡
30天35次卡
25天14次卡
30天10次卡
20天30次卡
3天6次卡
30天15次卡
7天3次立减卡
30天45次立减卡
7天3次卡
15天30次卡
15天12次卡
30天6次卡
7天10次卡


### 评估
topN召回商品与用户历史购买商品评估召回率

#### 评估实际购买过商品用户
因为本地跑的模型及预测，未连接公司数据库因此需要存储数据


In [ ]:
# 买过商品的用户
nt = test[test.td_is_buy_ride_card==1].reset_index(drop=True)
nt

In [ ]:
# 预测数据组装
test_nt = [{feat: nt[feat].values for feat in user_sparse_features  },
          {feat: nt[feat].values for feat in item_features}]

In [ ]:
# 模型预测
user_embs = user_embed_model.predict(test_nt[0])
item_embs = item_embed_model.predict(item_X)
user_embs = tf.squeeze(user_embs).numpy()
item_embs = tf.squeeze(item_embs).numpy()


In [ ]:
# cos距离
index = faiss.IndexFlatIP(item_embs.shape[1])
faiss.normalize_L2(item_embs)
index.add(np.array(item_embs))
faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 26)


In [ ]:
# j个用户i个召回商品，存储商品名
t_res = []
for j in range(nt.shape[0]):
    print('========')
    tmp_ls = []
    for i in I[j]: 
        tmp_ls.append(items.loc[i].product_name)
        print(items.loc[i].product_name)
        
    t_res.append(tmp_ls)

In [ ]:
# 索引列
nt['index_ls'] = I.tolist()

In [ ]:
# 索引对应的商品名列
def func(x):
    tmp_ls= []
    for i in x: 
        tmp_ls.append(items.loc[i].product_name)
    return tmp_ls
nt['name_ls'] = nt.index_ls.map(func)

In [ ]:
nt.to_csv('test_res_stable.csv')

#### 用户实际购买商品

In [ ]:
# 加载用户召回数据
test_res = pd.read_csv('test_res_stable.csv')
test_res.shape

In [ ]:
# 用户实际购买数据
uid = tuple(test_res.user_id.tolist())
true_df = get_df_from_csv_table(query_presto(f"""
       select user_id,array_join(array_distinct(array_agg(product_name)),'|') res from  
           (
           
         select   
                user_id,original_price,product_id,product_name,total_ride_card_times,ride_card_vaild_days,deduct_amt
            from ads.ads_bi_atm_abtest_coupon_ride_card_analysis_da -- undefined  
                where 1=1 
                and event_day BETWEEN '20230501' and  '20230614'
                and is_buy_ride_card = 1 
                and  user_id in {uid}    
                )
             group by user_id
"""))

In [ ]:
# 真实商品格式切分
true_df['true_col'] = true_df.res.map(lambda x:x.split('|'))
# 召回和实际关联
m1 = true_df.merge(test_res,on=['user_id'],how='inner')
# 召回商品列表
m1['name_ls'] = m1.name_ls.map(lambda x:eval(x))
# 真实商品数量
m1['total_nums'] = m1.true_col.map(lambda x:len(x))
# 召回和真实商品重合数量
m1['recall_nums'] = m1.apply(lambda x:len(set(x['true_col']).intersection(set(x['name_ls']))),axis=1)
# 召回率
m1['recall'] = m1.recall_nums/m1.total_nums

# 平均召回率 0.2
m1.recall.mean()

### 存在的问题/可改进的地方
- 特征处理粗糙：连续/数值分特征段过于随意
- 过采样可考虑smote等方法
- 实际使用时还需要考虑训练中未见过的/未知的类别特征值
    - 因此要给每个类别特征添加一个unknow类别
    - 一般是种类少的化为unknow
    - 没见过类别也提前处理为unknow
- 模型结构考虑加入连续特征
  - 参考model2和dssm2 
- 模型训练可能会提梯度消失，需要参考笔记修改
- 依赖tf2.11.0以上，python3.8以上
- 相似向量检索需使用faiss.normalize_L2处理才为cos距离
    - 余弦距离和余弦相似度好像没搞明白，后续需要再看下最后一个参考或是其他 
- 召回率低可能是因为样本的问题
    - 看的文章说负样本不能仅仅是曝光的，还得是其他的未曝光的，避免选择偏差，因此之前考虑：1.购卡用户is_buy_ride_card = 1为正例；2.非购卡骑行用户is_buy_ride_card = 0 and  is_ride=1 为负例；3.需要为负例补充商品特征值，负例商品不限于曝光未购买商品，应从所有商品中随机采样，避免选择偏差
    - 但是会存在负例用户当天没买，但其他天买了为其构造的“没买商品”，严重影响模型性能，因此做样本时应该避免用户买过的商品进行负采样
    - 另外买过的用户也会存在当天买了某个商品，其他天没买过构造了买过商品的负例，也应该避免出现
    - 总而言之就是用户一段周期内买过的商品可以为正样本，没买过的才给随机分为负样本，不能全盘随机
- 召回率低可能是因为loss的问题
    - point-wise loss只能分好坏，没有样本之间相互关系
    - 考虑用pair-wise loss，样本之间的相对顺序或者相对差异比单独的点对点差异更重要
- 模型参考
  - https://github.com/Funnyeah/recommend-tf2.0.git

In [25]:
tf.__version__

'2.11.0'

### 参考
- https://blog.51cto.com/u_15717393/5471448#32_DSSM_188
- https://zhuanlan.zhihu.com/p/345378441
- https://www.zhihu.com/question/426543628/answer/1631702878
- https://zhuanlan.zhihu.com/p/448680238
- https://blog.csdn.net/supergxt/article/details/121137489